In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.regularizers import l2


# Load data into a DataFrame
data = pd.read_csv('/Users/hardeesh/CSV/SF_Data.csv')

# Separate features (X) and target variable (y)
X = data.drop(columns=['Label'])
y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define your models
def shallow_model():
    model = Sequential([
        Dense(64, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(l2=0.0001)),
        BatchNormalization(),
        Dense(1, activation='sigmoid')
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = shallow_model()
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

# Define a function to convert the model to TensorFlow Lite format
def convert_to_tflite(model, X_train):
    # Create a concrete function from the model
    run_model = tf.function(lambda x: model(x))
    concrete_func = run_model.get_concrete_function(tf.TensorSpec(X_train.shape, tf.float32))

    # Convert the model to TensorFlow Lite format
    converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
    tflite_model = converter.convert()

    # Save the converted model to a .tflite file
    with open('shallow_model.tflite', 'wb') as f:
        f.write(tflite_model)

    print("Model has been converted to TensorFlow Lite format and saved as 'shallow_model.tflite'")

# Convert and save the model
convert_to_tflite(model, X_train)

Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 336us/step - accuracy: 0.8986 - loss: 0.2857 - val_accuracy: 0.8628 - val_loss: 0.3754
Epoch 2/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 333us/step - accuracy: 0.9416 - loss: 0.1643 - val_accuracy: 0.8117 - val_loss: 0.3424
Epoch 3/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 324us/step - accuracy: 0.9458 - loss: 0.1521 - val_accuracy: 0.9500 - val_loss: 0.1669
Epoch 4/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 322us/step - accuracy: 0.9522 - loss: 0.1360 - val_accuracy: 0.9359 - val_loss: 0.2507
Epoch 5/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 321us/step - accuracy: 0.9542 - loss: 0.1323 - val_accuracy: 0.9537 - val_loss: 0.1506
Epoch 6/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 322us/step - accuracy: 0.9550 - loss: 0.1316 - val_accuracy: 0.9370 - val_loss: 0.1959
Epoch 7/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 320us/step - accuracy: 0.9578 - loss: 0.1227 - val_accuracy: 0.9493 - val_loss: 0.1864
Epoch 8/10
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 3s 321us/step - accuracy: 0.9589 - loss: 0.11

Test Loss: 0.13682371377944946, Test Accuracy: 0.9617834687232971
Model has been converted to TensorFlow Lite format and saved as 'shallow_model.tflite'


2024-05-20 07:01:26.010577: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-05-20 07:01:26.010634: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
W0000 00:00:1716168686.036268  500917 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1716168686.036277  500917 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-20 07:01:26.045025: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:3064] Estimated count of arithmetic ops: 1.365 G  ops, equivalently 0.683 G  MACs


In [19]:
loaded_model = tf.keras.models.load_model('shallow_model.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the converted model to a .tflite file
with open('shallow_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model has been converted to TensorFlow Lite format and saved as 'shallow_model.tflite'")
# This code will train your model, save it, convert it to TensorFlow Lite format, and save the converted model as shallow_model.tflite.


AttributeError: 'Sequential' object has no attribute '_get_save_spec'

In [4]:
import numpy as np

def predict(input_data, model):
    # Ensure the input data is an array
    if not isinstance(input_data, np.ndarray):
        input_data = np.array(input_data)

    # Reshape the data if necessary
    if len(input_data.shape) == 1:
        input_data = np.expand_dims(input_data, axis=0)

    # Use the model to predict the output
    prediction = model.predict(input_data)

    # The output of the model is probabilities, convert to classes
    decoded_prediction = (prediction > 0.5).astype(int)

    return decoded_prediction

# Assuming you have trained the model already
model = shallow_model()
model.fit(X_train, y_train, epochs=1, batch_size=64, validation_split=0.2)

# Example input data
input_data = [26,29200,27,28960,64,64,27,1500,1500,52,17912000,34156,80]


# Use the function to predict the output
output = predict(input_data, model)

print(f"Predicted output: {output}")


8885/8885 ━━━━━━━━━━━━━━━━━━━━ 5s 506us/step - accuracy: 0.9128 - loss: 0.2455 - val_accuracy: 0.9182 - val_loss: 0.3002
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predicted output: [[1]]
